## Importing libraries

In [1]:
import pandas as pd
import os
import json

## Location of the directory containing data dictionary

In [2]:
directory_path = '../../data_ingest/ipeds/dictionaries'

In [3]:
table_names = ["ADM2022", "C2022DEP", "EFFY2022", "GR2022", "HD2022", "IC2022CAMPUSES"]

In [4]:
ADM2022_prefix = ""
C2022DEP_prefix = ""
EFFY2022_prefix =""
HD2022_prefix = ""
GR2022_prefix = ""
IC2022_CAMPUSES_prefix = ""

In [5]:
def is_convertible_to_int(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

## Creating JSON file for Chroma DB

In [6]:
for i, filename in enumerate(os.listdir(directory_path)):
    # Check if the item is a file (not a subdirectory)
    if os.path.isfile(os.path.join(directory_path, filename)):
        df = pd.read_excel(directory_path+"/"+filename,sheet_name = 'varlist')
        df1 = pd.read_excel(directory_path+"/"+filename,sheet_name = 'Frequencies')
        df2 = pd.read_excel(directory_path+"/"+filename,sheet_name = 'Introduction')
        dataset = df[['varname','format','DataType','varTitle']]
        dataset2 = df1[['varname','codevalue','valuelabel']]
        dataset.loc[:, 'DataType'] = dataset['DataType'].replace({'N': 'Numeric', 'A': 'Text'})
        tabledesc = str(str(df2['TableDescription'].iloc[0]).replace(',', ';').replace(':', '').replace('/',' or '))
        tableName = str(str(df2['TableName'].iloc[0]))
        del df, df1
        if table_names[i].lower() in filename:
            descr = ""
            for index, row in dataset.iterrows():
                varName= ''
                desc = ''
                if row['format'] in 'Disc':
                    varName = str(row['varname']).lower()
                    desc = str(row['varTitle']).replace(',', ';').replace(':', '').replace('/',' or ')
                    dtype = str(row['DataType'])
                    values = {}
                    
                    for index2, row2 in dataset2.iterrows():
                        if row['varname'] in row2['varname'] and len(row['varname']) == len(row2['varname']):
                            key = str(row2['valuelabel']).replace(',', ';').replace(':', '').replace("\\", "\\\\").replace("'","").replace('/',' or ')
                            if is_convertible_to_int(str(row2['codevalue'])):
                                try:
                                    values[key] = int(str(row2['codevalue']))
                                except ValueError:
                                    try:
                                        values[key] = float(str(row2['codevalue']))
                                    except ValueError:
                                        # values[key] = str(row2['codevalue'])
                                        print("Error in float value as well")
        
                            else:
                                 values[key] = str(row2['codevalue'])
                            
                    if len(descr) == 0:
                        descr += '{ "Table_Info": [ { "Table_Name": "'+ tableName + '", "Table_Description": '+ tabledesc +', "Columns": [ { "Column_Name": "' + varName + '", "Data_Type": "'+ dtype+ '", "Column_Description": "'+ desc +'", "encoded_values": "' + str(values) +'" }'
                    else:
                        descr += ', {"Column_Name": "' + varName + '", "Data_Type": "'+ dtype+ '", "Column_Description": "'+ desc  + '", "encoded_values": "' + str(values) +'" }'
                else:
                    varName = str(row['varname']).lower()
                    desc = str(row['varTitle']).replace(',', ';').replace(':', '').replace('/',' or ')
                    dtype = str(row['DataType'])
                    if len(descr) == 0:
                        descr += '{ "Table_Info": [ {"Table_Name": "'+ tableName + '", "Table_Description": '+ tabledesc +', "Columns": [ { "Column_Name": "' + varName + '", "Data_Type": "'+ dtype+ '", "Column_Description": "'+ desc+'" }'
                    else:
                        descr += ', {"Column_Name": "'+ varName + '", "Data_Type": "'+ dtype + '", "Column_Description": "'+ desc +'" }'
            descr += ']}]}'
            if i == 0:
                ADM2022_prefix += descr
            elif i ==1:
                C2022DEP_prefix += descr
            elif i ==2:
                EFFY2022_prefix += descr
            elif i ==3:
                GR2022_prefix += descr
            elif i ==4:
                HD2022_prefix += descr
            else:
                IC2022_CAMPUSES_prefix += descr

In [7]:
HD2022_prefix=HD2022_prefix.replace("\\", "\\\\")
ADM2022_prefix = ADM2022_prefix.replace("\\", "\\\\")

In [8]:
json_objects = [
    ADM2022_prefix,
C2022DEP_prefix,
EFFY2022_prefix,
HD2022_prefix,
GR2022_prefix,
IC2022_CAMPUSES_prefix,
]

In [9]:
json_file_path= r'tableinfo.json'

In [10]:
ls = []
for i in json_objects:
    ls.append(json.loads(i))

In [11]:
with open(json_file_path, "w") as json_file:
    json.dump(ls, json_file, indent=4)

## Creating JSON file for MongoDB

In [7]:
def to_double_quotes(s):
    return '"' + s.replace('"', '\\"') + '"'

In [7]:
descr = '[{ "Table_Info": ['
count_1=0
for i, filename in enumerate(os.listdir(directory_path)):
    # Check if the item is a file (not a subdirectory)
    if os.path.isfile(os.path.join(directory_path, filename)):
        df = pd.read_excel(directory_path+"/"+filename,sheet_name = 'varlist')
        df1 = pd.read_excel(directory_path+"/"+filename,sheet_name = 'Frequencies')
        df2 = pd.read_excel(directory_path+"/"+filename,sheet_name = 'Introduction')
        dataset = df[['varname','format','DataType','varTitle']]
        dataset2 = df1[['varname','codevalue','valuelabel']]
        dataset.loc[:, 'DataType'] = dataset['DataType'].replace({'N': 'Numeric', 'A': 'Text'})
        tabledesc = str(str(df2['TableDescription'].iloc[0]).replace(',', ';').replace(':', '').replace('/',' or '))
        tableName = str(str(df2['TableName'].iloc[0]))
        del df, df1
        if table_names[i].lower() in filename:
            for index, row in dataset.iterrows():
                varName= ''
                desc = ''
                if row['format'] in 'Disc':
                    varName = str(row['varname']).lower()
                    desc = str(row['varTitle']).replace(',', ';').replace(':', '').replace('/',' or ')
                    dtype = str(row['DataType'])
                    values = {}
                    for index2, row2 in dataset2.iterrows():
                        if row['varname'] in row2['varname'] and len(row['varname']) == len(row2['varname']):
                            key = str(row2['valuelabel']).replace(',', ';').replace(':', '').replace("\\",'').replace("'",'').replace('/',' or ')
                            key = repr(key).replace("'", '"')
                            if is_convertible_to_int(str(row2['codevalue'])) and dtype not in 'Text':
                                values[key] = float(str(row2['codevalue']))
                            else:
                                 values[key] = to_double_quotes(str(row2['codevalue'])) 
                    if tableName not in descr:
                        count_1+=1
                        descr += '{ "Table_Name": "'+ tableName + '", "Table_Description": '+ tabledesc +', "Columns": [ { "Column_Name": "' + varName + '", "Data_Type": "'+ dtype+ '", "Column_Description": "'+ desc +'", "encoded_values": ' + str(values).replace("'",'') + '}'
                    else:
                        descr += ', {"Column_Name": "' + varName + '", "Data_Type": "'+ dtype+ '", "Column_Description": "'+ desc  + '", "encoded_values": '  + str(values).replace("'",'') + '}'
                else:
                    varName = str(row['varname']).lower()
                    desc = str(row['varTitle']).replace(',', ';').replace(':', '').replace('/',' or ')
                    dtype = str(row['DataType'])
                    if tableName not in descr:
                        count_1+=1
                        descr += '{ "Table_Name": "'+ tableName + '", "Table_Description": '+ tabledesc +', "Columns": [ { "Column_Name": "' + varName + '", "Data_Type": "'+ dtype+ '", "Column_Description": "'+ desc+'" }'
                    else:
                        descr += ', {"Column_Name": "'+ varName + '", "Data_Type": "'+ dtype + '", "Column_Description": "'+ desc +'" }'
            if count_1 == len(table_names):
                descr += ']}'
            else:
                descr += ']},'
descr += ']}]'

In [11]:
ds = json.loads(descr)

In [12]:
json_file_path= r'tableinfo_new.json'

In [12]:
with open(json_file_path, "w") as json_file:
    json.dump(ds, json_file, indent=4)